# Initialise the project structure

In [ ]:
# @title Project environ

# @markdown You'll only need to setup a free account on NGROK
ngrok_auth_token = "" # @param {type:"string"}
# @markdown Follow the instructions here https://github.com/dsdanielpark/Bard-API
bard_api_key = "" # @param {type:"string"}
# @markdown Create an account at https://www.maptiler.com/ to get the Map Tiler API Key
mapteiler_auth_key = "" # @param {type:"string"}

import os
import sys

# Define your project folder name
drive_path = "/content/drive/My Drive"
project_folder_path = "Projects/Nasa Space Apps Challenge 2023 - HydroVision"
app_name_display = "What's My Water"
app_name = "whatsmywater"
output_path = f"{drive_path}/{project_folder_path}/outputs"

os.environ["drive_path"] = drive_path
os.environ["project_folder_path"] = project_folder_path
os.environ["app_name_display"] = app_name_display
os.environ["app_name"] = app_name
os.environ["output_path"] = output_path

os.environ["_ngrok_authtoken"] = ngrok_auth_token
os.environ["_mapteiler_auth_key"] = mapteiler_auth_key
os.environ["_bard_api_key"] = bard_api_key

sys.path.append(os.path.join(drive_path, project_folder_path))

In [ ]:
# @title Create the project folder

import os

# Create the project folder
if not os.path.exists(f'{drive_path}/{project_folder_path}'):
  !mkdir '{drive_path}/{project_folder_path}'

# Create subfolders and files
!mkdir '{drive_path}/{project_folder_path}/outputs'

!mkdir '{drive_path}/{project_folder_path}/src'
!touch '{drive_path}/{project_folder_path}/src/__init__.py'

!mkdir '{drive_path}/{project_folder_path}/static'
!mkdir '{drive_path}/{project_folder_path}/static/images'
!mkdir '{drive_path}/{project_folder_path}/static/js'
!touch '{drive_path}/{project_folder_path}/static/js/base_js.js'
!mkdir '{drive_path}/{project_folder_path}/static/css'
!touch '{drive_path}/{project_folder_path}/static/css/styles.css'
!touch '{drive_path}/{project_folder_path}/static/css/base_styles.css'

!mkdir '{drive_path}/{project_folder_path}/templates'
!mkdir '{drive_path}/{project_folder_path}/templates/macros'
!touch '{drive_path}/{project_folder_path}/templates/index.html'
!touch '{drive_path}/{project_folder_path}/templates/base.html'

In [ ]:
# @title Creating Cache files

import json
import os

with open(os.path.join(os.environ["output_path"], "cached_queries.json"), "w")as json_file:
  cache = { }

  json_file.write(json.dumps(cache, indent=2))

In [ ]:
# @title Install environment : Bard, Ngrok, Flask

!pip -q install bardapi

!pip -q install pyngrok

!pip -q install flask
!pip -q install SQLAlchemy
!pip install Flask-Session

In [ ]:
# @title Initialise authtoken

!ngrok authtoken {os.environ["_ngrok_authtoken"]}

In [ ]:
# @title Initialise a Tailwind project

%cd '{drive_path}/{project_folder_path}'
!npm init
!npm install tailwindcss
!npx tailwindcss init

# Run the project

In [ ]:
# @title Rebuild Tailwind

%cd '{drive_path}/{project_folder_path}'
!npx tailwindcss -i ./static/css/styles.css -o ./static/css/tailwind_styles.css

/content/drive/My Drive/Projects/Nasa Space Apps Challenge 2023 - HydroVision
Browserslist: caniuse-lite is outdated. Please run:
  npx update-browserslist-db@latest
  Why you should do it regularly: https://github.com/browserslist/update-db#readme

Rebuilding...

Done in 393ms.


In [ ]:
# @title Reload src
import sys
import types

def reload_package(root_module):
  package_name = root_module.__name__

  # get a reference to each loaded module
  loaded_package_modules = dict([
    (key, value) for key, value in sys.modules.items() if key.startswith(package_name) and isinstance(value, types.ModuleType
  )])

    # delete references to these loaded modules from sys.modules
  for key in loaded_package_modules:
    del sys.modules[key]

    # load each of the modules again;
    # make old modules share state with new modules
  for key in loaded_package_modules:
    print('loading %s' % key)
    newmodule = __import__(key)
    oldmodule = loaded_package_modules[key]
    oldmodule.__dict__.clear()
    oldmodule.__dict__.update(newmodule.__dict__)


def reload_src():
  import src
  from src import app_blueprint, home
  reload_package(src)

reload_src()

loading src
loading src.bard_prompt
loading src.home


In [ ]:
# @title Launch Ngrok agent
from pyngrok import ngrok

port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

 * ngrok tunnel "https://9d0c-34-136-73-152.ngrok-free.app" -> "http://127.0.0.1:5000"


In [ ]:
# @title Start Flask server

import os
import cv2
import json

from flask import Flask, render_template, redirect, request, session
from flask_session import Session

import src
from src import app_blueprint, home

static_dir = f'{drive_path}/{project_folder_path}/static'
template_dir = f'{drive_path}/{project_folder_path}/templates'

app = Flask(__name__, template_folder=template_dir, static_folder=static_dir)
# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"
Session(app)

app.register_blueprint(app_blueprint, url_prefix="/")

if __name__ == '__main__':
  app.run()

In [ ]:
# @title Kill Ngrok agent

ngrok.kill()